In [1]:
import gym
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from collections import deque
import random

c:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
env = gym.make('ALE/Tetris-v5', repeat_action_probability=0.0)
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

In [3]:
memory = deque(maxlen=2000)
batch_size = 64
gamma = 0.95  # Discount factor
epsilon = 1.0  # Exploration rate
epsilon_min = 0.01
epsilon_decay = 0.995
target_update_counter = 0
update_target_every = 5  # Update target network every 5 episodes
episodes = 1000
max_steps_per_episode = 1000
learning_rate = 0.001

In [4]:
def build_model():
    model = Sequential()
    model.add(Dense(24, input_dim=state_size, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(action_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(learning_rate))
    return model

# Initialize DQN and target DQN
dqn = build_model()
target_dqn = build_model()

C:\Users\bsilver3\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
def remember(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))

def replay(batch_size):
    if len(memory) < batch_size:
        return
    minibatch = random.sample(memory, batch_size)
    for state, action, reward, next_state, done in minibatch:
        target = reward
        if not done:
            target = (reward + gamma * np.amax(target_dqn.predict(next_state)[0]))
        target_f = dqn.predict(state)
        target_f[0][action] = target
        dqn.fit(state, target_f, epochs=1, verbose=0)

In [6]:
for episode in range(episodes):
    state = env.reset()
    done = False
    total_reward = 0

    for step in range(max_steps_per_episode):
        if np.random.rand() <= epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(dqn.predict(state))

        next_state, reward, done, _ = env.step(action)

        remember(state, action, reward, next_state, done)

        total_reward += reward
        state = next_state

        replay(batch_size)

        if done:
            break

    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    target_update_counter += 1
    if target_update_counter >= update_target_every:
        target_dqn.set_weights(dqn.get_weights())
        target_update_counter = 0

    print(f'Episode {episode + 1}/{episodes}, Total Reward: {total_reward}')

C:\Users\bsilver3\AppData\Roaming\Python\Python39\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


ValueError: too many values to unpack (expected 4)